# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [2]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns


Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [3]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
(n_students, n_columns) = student_data.shape 
n_features = n_columns - 1  # don't count the label as a feature
n_passed = len(student_data[(student_data['passed']=='yes')])
n_failed = len(student_data[(student_data['passed']=='no')])
grad_rate = 100.0*n_passed/n_students
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

# easiest classifier, just say 'yes' student will 'pass'
just_say_yes_precision = 1.0*n_passed/n_students
just_say_yes_recall = 1.0*n_students/n_students
just_say_yes_f1 = 2*just_say_yes_precision*just_say_yes_recall/(just_say_yes_precision+just_say_yes_recall)
print '' 
print "Just say yes precision: %4.2f" % just_say_yes_precision
print "Just say yes recall: %4.2f" % just_say_yes_recall
print "Just say yes F1: %4.2f" % just_say_yes_f1

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%

Just say yes precision: 0.67
Just say yes recall: 1.00
Just say yes F1: 0.80


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [156]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [157]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [158]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

s0 = pd.Series([i for i in xrange(num_all)]).sample(num_all, replace=False, random_state=0)

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
X_train=X_all[X_all.index.isin(s0.head(num_train))]
X_test=X_all[X_all.index.isin(s0.tail(num_test))]
y_train=y_all[y_all.index.isin(s0.head(num_train))]
y_test=y_all[y_all.index.isin(s0.tail(num_test))]

print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data


Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [159]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)

# TODO: Choose a model, import it and instantiate an object
import sklearn.tree
clf = sklearn.tree.DecisionTreeClassifier()

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
#print clf  # you can inspect the learned model by printing it

Training DecisionTreeClassifier...
Done!
Training time (secs): 0.003


In [160]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target, pos_label='yes'):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    return f1_score(target.values, y_pred, pos_label=pos_label)

train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.001
F1 score for training set: 1.0


In [162]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.761194029851


In [163]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant

train_predict(clf, X_train, y_train, X_test,y_test)

X_train_200=X_all[X_all.index.isin(s0.head(200))]
y_train_200=y_all[y_all.index.isin(s0.head(200))]
train_predict(clf, X_train_200, y_train_200, X_test,y_test)

X_train_100=X_all[X_all.index.isin(s0.head(100))]
y_train_100=y_all[y_all.index.isin(s0.head(100))]
train_predict(clf, X_train_100, y_train_100, X_test,y_test)



------------------------------------------
Training set size: 300
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.003
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 1.0
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.757575757576
------------------------------------------
Training set size: 200
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 1.0
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.779411764706
------------------------------------------
Training set size: 100
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 

In [164]:
# TODO: Train and predict using two other models
import sklearn.neighbors
kn_clf = sklearn.neighbors.KNeighborsClassifier()

train_predict(kn_clf, X_train, y_train, X_test,y_test)
train_predict(kn_clf, X_train_200, y_train_200, X_test, y_test)
train_predict(kn_clf, X_train_100, y_train_100, X_test, y_test)

print "\n===\n"

import sklearn.svm
svm_clf = sklearn.svm.SVC()

train_predict(svm_clf, X_train, y_train, X_test,y_test)
train_predict(svm_clf, X_train_200, y_train_200, X_test, y_test)
train_predict(svm_clf, X_train_100, y_train_100, X_test, y_test)


------------------------------------------
Training set size: 300
Training KNeighborsClassifier...
Done!
Training time (secs): 0.002
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.008
F1 score for training set: 0.858513189448
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.005
F1 score for test set: 0.777777777778
------------------------------------------
Training set size: 200
Training KNeighborsClassifier...
Done!
Training time (secs): 0.001
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.005
F1 score for training set: 0.805970149254
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.004
F1 score for test set: 0.805555555556
------------------------------------------
Training set size: 100
Training KNeighborsClassifier...
Done!
Training time (secs): 0.001
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.004
F1 score for training

## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

In [191]:
# TODO: Fine-tune your model and report the best F1 score
n_test_yes = len(y_test[y_test=='yes'])
n_test_no  = len(y_test[y_test=='no'])
# precison just say 'yes'
prec = 1.0*n_test_yes/(n_test_yes+n_test_no)
# recall just say 'yes'
recall = 1.00
# f1 just say 'yes'
f1 = 2*prec*recall/(prec+recall)
print "baseline just say yes on test set: prec=%4.2f, recall=%4.2f, f1=%4.2f " % (prec, recall, f1)

import sklearn.grid_search
import sklearn.metrics

# default F1 scoreing depends on 1,0, need to let know our positive label is 'yes'
yes_f1_scorer = sklearn.metrics.make_scorer(f1_score, pos_label='yes')

params = [
        {'kernel': ['poly'], 'degree': [2, 3, 4, 5], 'gamma': [0.001, 0.0001], 'C': [1, 10, 100, 1000]},
        {'kernel': ['rbf', 'sigmoid'], 'gamma': [0.001, 0.0001], 'C': [1, 10, 100, 1000]},
        {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}
    ]

grid_search = sklearn.grid_search.GridSearchCV(
    sklearn.svm.SVC(), 
    param_grid=params, 
    scoring=yes_f1_scorer)
grid_search.fit(X_train, y_train)

print grid_search.best_params_
print grid_search.best_estimator_
print grid_search.best_score_

final_clf = grid_search.best_estimator_
print "F1 score for training set: {}".format(predict_labels(final_clf, X_train, y_train))
print "F1 score for test set: {}".format(predict_labels(final_clf, X_test, y_test))

y_pred = final_clf.predict(X_test)
print sklearn.metrics.confusion_matrix(y_test, y_pred)

baseline just say yes on test set: prec=0.74, recall=1.00, f1=0.85 
{'kernel': 'sigmoid', 'C': 100, 'gamma': 0.0001}
SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.0001, kernel='sigmoid',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
0.797762956291
Predicting labels using SVC...
Done!
Prediction time (secs): 0.004
F1 score for training set: 0.798319327731
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.867924528302
[[ 5 20]
 [ 1 69]]


In [192]:
no_f1_scorer = sklearn.metrics.make_scorer(f1_score, pos_label='no')

params = [
        {'kernel': ['poly'], 'degree': [2, 3, 4, 5], 'gamma': [0.001, 0.0001], 'C': [1, 10, 100, 1000]},
        {'kernel': ['rbf', 'sigmoid'], 'gamma': [0.001, 0.0001], 'C': [1, 10, 100, 1000]},
        {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}
    ]

no_grid_search = sklearn.grid_search.GridSearchCV(
    sklearn.svm.SVC(), 
    param_grid=params, 
    scoring=no_f1_scorer)
no_grid_search.fit(X_train, y_train)

print no_grid_search.best_params_
print no_grid_search.best_estimator_
print no_grid_search.best_score_

no_clf = no_grid_search.best_estimator_
print "F1 score for training set: {}".format(predict_labels(no_clf, X_train, y_train,'no'))
print "F1 score for test set: {}".format(predict_labels(no_clf, X_test, y_test,'no'))

no_y_pred = no_clf.predict(X_test)
print sklearn.metrics.confusion_matrix(y_test, no_y_pred)

{'kernel': 'poly', 'C': 1000, 'gamma': 0.001, 'degree': 2}
SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=2, gamma=0.001, kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
0.462738345446
Predicting labels using SVC...
Done!
Prediction time (secs): 0.004
F1 score for training set: 0.755319148936
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.341463414634
[[ 7 18]
 [ 9 61]]


In [189]:
# default F1 scoreing depends on 1,0, need to let know our positive label is 'yes'

dt_params = [
        {'max_depth': (1,2,3,4,5,6,7,8,9,10)}
    ]

dt_grid_search = sklearn.grid_search.GridSearchCV(
    sklearn.tree.DecisionTreeClassifier(), 
    param_grid=dt_params, 
    scoring=no_f1_scorer)
dt_grid_search.fit(X_train, y_train)

print dt_grid_search.best_params_
print dt_grid_search.best_estimator_
print dt_grid_search.best_score_

dt_clf = dt_grid_search.best_estimator_
print "F1 score for training set: {}".format(predict_labels(dt_clf, X_train, y_train, 'no'))
print "F1 score for test set: {}".format(predict_labels(dt_clf, X_test, y_test, 'no'))

dt_y_pred = dt_clf.predict(X_test)
print sklearn.metrics.confusion_matrix(y_test, dt_y_pred)

{'max_depth': 1}
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
0.5109963232
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.523255813953
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.341463414634
[[ 7 18]
 [ 9 61]]


In [190]:
knn_params = [
        {'algorithm': ['auto','ball_tree','kd_tree','brute'], 'p': [1,2,3,4]}
    ]

knn_grid_search = sklearn.grid_search.GridSearchCV(
    sklearn.neighbors.KNeighborsClassifier(), 
    param_grid=knn_params, 
    scoring=no_f1_scorer)
knn_grid_search.fit(X_train, y_train)

print knn_grid_search.best_params_
print knn_grid_search.best_estimator_
print knn_grid_search.best_score_

knn_clf = knn_grid_search.best_estimator_
print "F1 score for training set: {}".format(predict_labels(knn_clf, X_train, y_train,'no'))
print "F1 score for test set: {}".format(predict_labels(knn_clf, X_test, y_test,'no'))

knn_y_pred = knn_clf.predict(X_test)
print sklearn.metrics.confusion_matrix(y_test, knn_y_pred)

{'p': 3, 'algorithm': 'brute'}
KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=3,
           weights='uniform')
0.448196721311
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.066
F1 score for training set: 0.641711229947
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.024
F1 score for test set: 0.297872340426
[[ 7 18]
 [15 55]]
